In [23]:
from lxml import etree
import numpy as np

In [24]:
tree = etree.parse('rekordbox_bak_20240109.xml')

In [25]:
for child in tree.getroot().getchildren():
    print(child.tag, child.getchildren())

PRODUCT []
COLLECTION [<Element TRACK at 0x112ad5e80>, <Element TRACK at 0x112ad7e80>, <Element TRACK at 0x112ad6380>, <Element TRACK at 0x112ad5a80>, <Element TRACK at 0x112ad7040>, <Element TRACK at 0x112ad4240>, <Element TRACK at 0x112ad4580>, <Element TRACK at 0x112ae3600>, <Element TRACK at 0x112ae1b80>, <Element TRACK at 0x112ad4480>, <Element TRACK at 0x112ae0a00>, <Element TRACK at 0x112ae18c0>, <Element TRACK at 0x112ae0f80>, <Element TRACK at 0x112ae0680>, <Element TRACK at 0x112ae0480>, <Element TRACK at 0x112ae33c0>, <Element TRACK at 0x112ae00c0>, <Element TRACK at 0x112ae0400>, <Element TRACK at 0x112ae3fc0>, <Element TRACK at 0x112ae0980>, <Element TRACK at 0x112ae3f80>, <Element TRACK at 0x112ae0840>, <Element TRACK at 0x112ae3bc0>, <Element TRACK at 0x112ae1040>, <Element TRACK at 0x112ae2040>, <Element TRACK at 0x112ae29c0>, <Element TRACK at 0x112ae0d00>, <Element TRACK at 0x112ae3440>, <Element TRACK at 0x112ae1bc0>, <Element TRACK at 0x112ae1180>, <Element TRACK at

In [26]:
tracks = tree.getroot().xpath('//COLLECTION/TRACK')

for track in tracks:
    track_name = track.attrib['Name']
    bpms = track.xpath('TEMPO/@Bpm')
    bpms = [float(i) for i in bpms]

    if len(bpms) == 0:
        print(f"{track_name} not analyzed, skipping")
        continue
    if max(bpms) != np.mean(bpms):
        print(f'Warning - Track has multiple BPMs: {track_name} ')
    bpm = bpms[0]

    if "PROCESSED" in track.xpath('POSITION_MARK/@Name'):
        print("Track already processed, skipping")
        continue

    cues = track.xpath('POSITION_MARK')
    print(cues)
    if len(cues) > 3:
        print(f"SKIPPING - Unprocessed track has >3 cue points: {track_name}")
        continue
    if len(cues) == 0:
        print(f"SKIPPING - Unprocessed track has no cue points: {track_name}")
        continue


    #RENAME EXISTING CUES
    def get_time_attrib(elem):
        return float(elem.attrib['Start'])

    cues.sort(key=get_time_attrib)
    for i, elem in enumerate(cues):
        elem.attrib['Name'] = f"DROP {i+1}"


    #ASSUMES FIRST CUE IS AT THE DROP    
    #get cue timestamp
    drop_timestamp = float(min(cues, key=get_time_attrib).attrib['Start'])
    #create new cue points 8/16/24/32 bars before
    intervals = [8,16,24,32]
    beats = [x*4 for x in intervals]
    seconds_per_beat = 60/bpm

    new_cue_timestamps = [drop_timestamp - (x * seconds_per_beat) for x in beats]
    CUE_TYPES = ["0", "1", "2", "3"] #Makes the cues A, B, C, D hotcues
    for i in range(len(new_cue_timestamps)):
        ts = new_cue_timestamps[i]
        if ts > 0:
            ts = format(ts, '.3f')
            new_cue = etree.Element("POSITION_MARK", Name=f"{str(intervals[i])} BEFORE DROP", Type="0", Start=ts, Num=CUE_TYPES[i])
            track.insert(1, new_cue)
    print(new_cue_timestamps)
    #make sure time >0 on all new cues

[]
SKIPPING - Unprocessed track has no cue points: Slayyyter - Daddy AF (Wuki Remix)
[]
SKIPPING - Unprocessed track has no cue points: A.G. COOK - DROP FM (FEAT. HANNAH DIAMOND) HWLS_RETWERK
[]
SKIPPING - Unprocessed track has no cue points: umru + laura les—popular (Dylan Brady remix)
[]
SKIPPING - Unprocessed track has no cue points: Mii theme dub
[]
SKIPPING - Unprocessed track has no cue points: Sweat (SOPHIE Remix)
[]
SKIPPING - Unprocessed track has no cue points: : BUBBYTAFFY ! :
[<Element POSITION_MARK at 0x112aeb300>]
[82.189, 66.189, 50.18899999999999, 34.18899999999999]
[<Element POSITION_MARK at 0x112aeb900>]
[0.03399899974993659, -15.970002000500127, -31.97400300075019, -47.97800400100026]
[]
SKIPPING - Unprocessed track has no cue points: Hey QT
[]
SKIPPING - Unprocessed track has no cue points: 8 now
[]
SKIPPING - Unprocessed track has no cue points: Nautilus Shell Hoarding (w/ Ari Liloia)
[]
SKIPPING - Unprocessed track has no cue points: Ram It Down (feat. Mood Killer

In [27]:
tree.write('output.xml', pretty_print=True, encoding="utf-8", xml_declaration=True)